In [18]:
import re
from functools import partial

def AcceptReject(part, workflows, state):
    index = 0 
    rules, destination = workflows[state]
    while index < len(rules):
        property, fn, target = rules[index]
        if fn(part[property]):
            if target == "A":
                return sum([part[k] for k in part.keys()])
            if target == "R":
                return 0
            return AcceptReject(part, workflows, target)
        index += 1
    if destination == "A":
        return sum([part[k] for k in part.keys()])
    elif destination == "R":
        return 0
    return AcceptReject(part, workflows, destination)

def lt(target, value):
    return value < target
def gt(target, value):
    return value > target

for test in [True, False]:
    with open("test.txt" if test else "today.txt") as file:
        lines = [line.rstrip() for line in file] 

    ln = 0
    rules = True
    workflows = {}
    partone = 0
    while ln < len(lines):
        line = lines[ln]
        ln += 1
        if line == "": 
            rules = False
            continue
        if rules:
            p = re.compile("(\w+){(.+)}")
            m = p.match(line)
            name = m.group(1).strip()
            workflow = m.group(2).strip()
            rs = [] 
            destination = None
            for item in workflow.split(","):
                if ':' in item:
                    op, fn = ('<',lt) if '<' in item else ('>', gt)
                    property = item[:item.find(op)]
                    colon = item.find(':')
                    value = int(item[item.find(op)+1:colon])
                    f = partial(fn, value)
                    target = item[colon+1:]
                    rs.append((property, f, target))
                else:
                    destination = item.strip()
            workflows[name] = (rs, destination)
        else:
            xmas = {}
            part = line[1:-1].split(",")
            for p in part:
                ps = p.split("=")
                xmas[ps[0]] = int(ps[1])
            partone += AcceptReject(xmas, workflows, "in")

    if test:
        assert partone == 19114
    else:
        print("Part One:", partone)

    

    
    

Part One: 395382
